In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from io import StringIO

def calculate_trajectory_deviation(traj1, traj2):
    """
    计算两条轨迹的位置和姿态偏差。
    轨迹应为 (N, 6) 的形状，其中 N 是点数，6是 (x, y, z, roll, pitch, yaw)。
    """
    # 分离位置 (前3列) 和姿态 (后3列)
    pos1, rot1 = traj1[:, :3], traj1[:, 3:]
    pos2, rot2 = traj2[:, :3], traj2[:, 3:]

    # 计算每对对应点之间的欧几里得距离
    positional_errors = np.linalg.norm(pos1 - pos2, axis=1)
    # 计算平均绝对位置误差
    positional_mae = np.mean(positional_errors)

    # 计算姿态角度的绝对差值
    rotational_errors = np.abs(rot1 - rot2)
    # 计算平均绝对姿态误差
    rotational_mae = np.mean(rotational_errors)

    return positional_mae, rotational_mae, positional_errors

def visualize_trajectories(traj1, traj2, positional_errors):
    """
    使用 matplotlib 可视化两条3D轨迹。
    """
    pos1 = traj1[:, :3]
    pos2 = traj2[:, :3]

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # 绘制原始轨迹
    ax.plot(pos1[:, 0], pos1[:, 1], pos1[:, 2], 'o-', label='Original Trajectory', color='blue')
    # 绘制推理轨迹
    ax.plot(pos2[:, 0], pos2[:, 1], pos2[:, 2], 'o-', label='Inferred Trajectory', color='red')

    # 绘制对应点之间的误差线
    for i in range(len(pos1)):
        ax.plot([pos1[i, 0], pos2[i, 0]], [pos1[i, 1], pos2[i, 1]], [pos1[i, 2], pos2[i, 2]],
                '--', color='gray', linewidth=0.8)

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Trajectory Comparison')
    ax.legend()
    plt.show() # 在 Notebook 中，这会直接在下方渲染出图像

# 1. 加载推理得到的轨迹数据
inferred_actions_path = '/home/adminroot/lxx/openpi/code/openpi/test/action_chunk.txt'

with open(inferred_actions_path, 'r') as f:
    # 读取文件内容，移除括号和换行符，使其成为一个由空格分隔的长字符串
    content = f.read().replace('[', '').replace(']', '').replace('\n', ' ')

# 将字符串分割成数字列表，转换为numpy数组，然后重塑为 (10, 7)
inferred_actions_raw = np.array(content.split(), dtype=float).reshape(10, 7)

# 忽略第7维数据
inferred_actions = inferred_actions_raw[:, :6]

# 2. 加载原始轨迹数据
original_actions = np.array([
    [
      0.036876101288518925,
      0.004070143430655884,
      -0.0013488432204023766,
      0.13572300192750397,
      0.08339984778797316,
      -0.566674593515589
    ],
    [
      0.10125030036575523,
      0.008512219905303126,
      -0.0037764461264646358,
      -0.07832043486927875,
      0.017585419593046936,
      -1.2099772652121419
    ],
    [
      0.19191249776155864,
      0.011888579566873544,
      -0.00727426463628982,
      -0.0506325305561357,
      0.0,
      -2.5397376487488157
    ],
    [
      0.3183088757560593,
      0.017202506371169407,
      -0.0908050819004035,
      0.12608159057919366,
      0.07857914211382422,
      -3.6974457656158335
    ],
    [
      0.48471868292169,
      0.018667764916854172,
      -0.1188008005698685,
      0.40105435637341214,
      0.10000000000002274,
      -4.552091335755335
    ],
    [
      0.7036599173005958,
      0.018694293371259815,
      -0.20455542946368208,
      0.3149314994916421,
      0.17699590342994043,
      -4.733075202363617
    ],
    [
      0.9660358837525309,
      0.020482839535330706,
      -0.3135314999030428,
      0.04236920936557742,
      0.12448196767132913,
      -4.204448976063089
    ],
    [
      1.2709258668099548,
      0.020996216244995446,
      -0.42252728351604313,
      0.11360791371131906,
      0.10000000000002274,
      -3.599021380284213
    ],
    [
      1.6217891766850894,
      0.01860037893627623,
      -0.4641771134630425,
      0.09783868882574098,
      0.10000000000002274,
      -3.2909059673120633
    ],
    [
      2.0100608871845975,
      0.013484811382303044,
      -0.5502218055772891,
      0.13936969171593216,
      0.10000000000002274,
      -2.739950504812716
    ]
])

# 3. 计算偏差
pos_mae, rot_mae, pos_errors = calculate_trajectory_deviation(original_actions, inferred_actions)

print("Trajectory Deviation Analysis:")
print("------------------------------")
# 平均绝对位置误差 
print(f"Average Positional MAE (Mean Absolute Error): {pos_mae:.4f}")
# 平均绝对姿态误差
print(f"Average Rotational MAE (Mean Absolute Error): {rot_mae:.4f}")
print("\nPositional error for each point:")
for i, err in enumerate(pos_errors):
    print(f"  Point {i+1}: {err:.4f}")

# 4. 可视化轨迹
visualize_trajectories(original_actions, inferred_actions, pos_errors)